### __Notebook Overview__ 

The Notebook's aim is to Create a Natural Langauge processing model 
using supervised learning Approach.we will outline the process below 

##### __0-6 Preprocessing Data__
##### __7-9 Model Building__ 
##### __10 Model Selection__
##### __11. Testing Model__ 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing

##Data Cleaning
import nltk #
import string #to remove all punctuation marks in the corpus/ document
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk import SnowballStemmer, PorterStemmer, LancasterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re

from sklearn.model_selection import train_test_split

ps = nltk.PorterStemmer()

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, accuracy_score
# set plot style
sns.set()

### __0.Loading Data__ 

In [2]:
df_train = pd.read_csv("train_set.csv")
df_test = pd.read_csv('test_set.csv')
df_sample = pd.read_csv('sample_submission.csv')

### __0.1.View Data__

In [3]:
pd.set_option("display.max_colwidth", -1)

C:\Users\NeoSebanze\AppData\Local\Temp\ipykernel_17668\2475433900.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


In [4]:
df_train.shape

(33000, 2)

### __1.Drop duplicates__ 

In [5]:
df_train.drop_duplicates(subset='text',inplace=True)
df_train.shape

(29948, 2)

In [6]:
df_train.head(4)

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko axhasa ulawulo lwesininzi kunye nokuthath inxaxheba kwabafazi ezi ziquka phakathi kwezinye zazo ikomishoni yokulingana ngokwesini ikomishoni yamalungelo oluntu lomzantsi afrika
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi naphi na kwisebe ngokusekwe kwiimfuno zokusebenza zalo emva kokubonana nomsebenzi kunye okanye imanyano yakhe ukuba ulandulo lomntu onjalo alufanelekanga i-dha mayibize uncedo olufanelekileyo elungelweni layo
2,eng,the province of kwazulu-natal department of transport invites tenders from established contractors experienced in bridge construction for the construction of the kwajolwayo tugela river pedestrian bridge near tugela ferry the duration of the project will be months
3,nso,o netefatša gore o ba file dilo ka moka tše le dumelelanego ka tšona mohlala maleri a magolo a a šomišwago go fihlelela meagong e metelele scaffolds a a bolokegilego lefelo la maleba la go šomela go phela gabotse bjbj


### __2.Null Values__

In [7]:
df_train.isnull().sum()

lang_id    0
text       0
dtype: int64

### __3.Clean Data__

In [8]:
def clean(df):
    
    #1.remove short words
    df['text'] = df['text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
    
    #3.Convert all uppercase to lower
    df['text'] = df['text'].str.lower()
    
    #4.punctuations , numbers and special characters except for the dash -
    df['text'] = df['text'].str.replace("[^a-zA-Z-]", " ")
    
    return df

In [9]:
clean(df_train)

C:\Users\NeoSebanze\AppData\Local\Temp\ipykernel_17668\3637475032.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace("[^a-zA-Z-]", " ")


,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko axhasa ulawulo lwesininzi kunye nokuthath inxaxheba kwabafazi ziquka phakathi kwezinye zazo ikomishoni yokulingana ngokwesini ikomishoni yamalungelo oluntu lomzantsi afrika
1,xho,i-dha kuba nobulumko bokubeka umsebenzi naphi kwisebe ngokusekwe kwiimfuno zokusebenza zalo emva kokubonana nomsebenzi kunye okanye imanyano yakhe ukuba ulandulo lomntu onjalo alufanelekanga i-dha mayibize uncedo olufanelekileyo elungelweni layo
2,eng,province kwazulu-natal department transport invites tenders from established contractors experienced bridge construction construction kwajolwayo tugela river pedestrian bridge near tugela ferry duration project will months
3,nso,netefat a gore file dilo moka dumelelanego t ona mohlala maleri magolo omi wago fihlelela meagong metelele scaffolds bolokegilego lefelo maleba omela phela gabotse bjbj
4,ven,khomishini ndinganyiso mbeu maana mulayo khomishini ndinganyiso mbeu thetshelesa mbilaelo dzine tshimbilelana tshialula mbeu nahone ivhea foramu thungo mulayo ndinganyiso
...,...,...
32994,eng,manuel marin ill-fated debt sources very little from fiscu would like take this relief initiative marin countries opportunity wish bongi proposal write notwithstanding best luck
32995,tsn,popo dipolateforomo tlisa boeteledipele saenseng lefatse rona kgato kgolo pele popont hwa thekeniki biothekeniki motsamaisi thekenoloji biotekeniki lefapha saense thekenoloji durham
32997,eng,closing date submission completed tenders august late submissions will considered submissions must enclosed sealed envelope addressed chief executive officer above address
32998,xho,nawuphina umntu ofunyenwe enetyala phantsi kwalo mthetho ujongene ujungene nesohlwayo sokudliwa imali okanye ukuvalelwa ixesha elingadluliyo kwiinyanga ezintandathu okanye kuzo zombini isohlwayo sokudliwa imali okanye ukuvalelwa


### __4.Tokenization__

In [10]:
tokenized_text = df_train['text'].apply(lambda x: x.split())

tokenized_text.head()

0    [umgaqo-siseko, wenza, amalungiselelo, kumaziko, axhasa, ulawulo, lwesininzi, kunye, nokuthath, inxaxheba, kwabafazi, ziquka, phakathi, kwezinye, zazo, ikomishoni, yokulingana, ngokwesini, ikomishoni, yamalungelo, oluntu, lomzantsi, afrika]                                   
1    [i-dha, kuba, nobulumko, bokubeka, umsebenzi, naphi, kwisebe, ngokusekwe, kwiimfuno, zokusebenza, zalo, emva, kokubonana, nomsebenzi, kunye, okanye, imanyano, yakhe, ukuba, ulandulo, lomntu, onjalo, alufanelekanga, i-dha, mayibize, uncedo, olufanelekileyo, elungelweni, layo]
2    [province, kwazulu-natal, department, transport, invites, tenders, from, established, contractors, experienced, bridge, construction, construction, kwajolwayo, tugela, river, pedestrian, bridge, near, tugela, ferry, duration, project, will, months]                           
3    [netefat, a, gore, file, dilo, moka, dumelelanego, t, ona, mohlala, maleri, magolo, omi, wago, fihlelela, meagong, metelele, scaffolds, bolokegilego, le

### __5.Stop words__

###### __REMOVE STOP WORDS WHEN THE LANGUAGE IS ENGLISH__

In [11]:
nltk.download()
stopword = nltk.corpus.stopwords.words('english')

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [12]:
def remove_stopwords(text):
    text=[word for word in text if word not in stopword]
    return text
stop_words = tokenized_text.apply(lambda x: remove_stopwords(x))
stop_words.head()

0    [umgaqo-siseko, wenza, amalungiselelo, kumaziko, axhasa, ulawulo, lwesininzi, kunye, nokuthath, inxaxheba, kwabafazi, ziquka, phakathi, kwezinye, zazo, ikomishoni, yokulingana, ngokwesini, ikomishoni, yamalungelo, oluntu, lomzantsi, afrika]                                   
1    [i-dha, kuba, nobulumko, bokubeka, umsebenzi, naphi, kwisebe, ngokusekwe, kwiimfuno, zokusebenza, zalo, emva, kokubonana, nomsebenzi, kunye, okanye, imanyano, yakhe, ukuba, ulandulo, lomntu, onjalo, alufanelekanga, i-dha, mayibize, uncedo, olufanelekileyo, elungelweni, layo]
2    [province, kwazulu-natal, department, transport, invites, tenders, established, contractors, experienced, bridge, construction, construction, kwajolwayo, tugela, river, pedestrian, bridge, near, tugela, ferry, duration, project, months]                                       
3    [netefat, gore, file, dilo, moka, dumelelanego, ona, mohlala, maleri, magolo, omi, wago, fihlelela, meagong, metelele, scaffolds, bolokegilego, lefelo, 

### __6.Lemmatisation__

In [13]:
def train_lemma(words, lemmatizer):
    return [lemmatizer.lemmatize(word) for word in words]  

lemmatizer = WordNetLemmatizer()

df_lemma = stop_words.apply(train_lemma, args=(lemmatizer, ))

### __7.Train Test Split__

In [14]:
X = df_train.text
y = df_train['lang_id']

In [15]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

### __8.1 Appplying Logistic Regression__

In [16]:
#Appplying Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

lr = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', LogisticRegression(multi_class='ovr')),
              ])
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)

In [17]:
print(f"Accuracy is : {accuracy_score(y_pred,y_test)}")
print("\n\nClassification Report:\n\n", classification_report(y_test, y_pred))

Accuracy is : 0.9914858096828046


Classification Report:

               precision    recall  f1-score   support

         afr       1.00      1.00      1.00       553
         eng       1.00      1.00      1.00       603
         nbl       0.98      0.96      0.97       473
         nso       0.99      1.00      1.00       559
         sot       1.00      1.00      1.00       606
         ssw       0.99      0.99      0.99       481
         tsn       0.99      0.99      0.99       543
         tso       1.00      1.00      1.00       548
         ven       1.00      1.00      1.00       509
         xho       0.98      0.99      0.99       503
         zul       0.97      0.97      0.97       612

    accuracy                           0.99      5990
   macro avg       0.99      0.99      0.99      5990
weighted avg       0.99      0.99      0.99      5990



### __8.2 Multinomial naive bayes__

In [18]:
from sklearn.naive_bayes import MultinomialNB


mn = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clr', MultinomialNB()),
              ])
mn.fit(X_train,y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clr', MultinomialNB())])

In [19]:
y_pred = mn.predict(X_test)

print(f"Accuracy is : {accuracy_score(y_pred,y_test)}")
print("\n\nClassification Report:\n\n", classification_report(y_test, y_pred))

Accuracy is : 0.9979966611018364


Classification Report:

               precision    recall  f1-score   support

         afr       1.00      1.00      1.00       553
         eng       0.99      1.00      1.00       603
         nbl       1.00      0.99      1.00       473
         nso       1.00      1.00      1.00       559
         sot       1.00      1.00      1.00       606
         ssw       1.00      1.00      1.00       481
         tsn       1.00      1.00      1.00       543
         tso       1.00      1.00      1.00       548
         ven       1.00      1.00      1.00       509
         xho       1.00      0.99      1.00       503
         zul       0.99      1.00      0.99       612

    accuracy                           1.00      5990
   macro avg       1.00      1.00      1.00      5990
weighted avg       1.00      1.00      1.00      5990



### __8.3 Decision tree__

In [20]:
from sklearn.tree import DecisionTreeClassifier


tree = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clr', DecisionTreeClassifier()),
              ])
tree.fit(X_train,y_train)
y_pred = tree.predict(X_test)

In [21]:
print(f"Accuracy is : {accuracy_score(y_pred,y_test)}")
print("\n\nClassification Report:\n\n", classification_report(y_test, y_pred))

Accuracy is : 0.8958263772954925


Classification Report:

               precision    recall  f1-score   support

         afr       0.99      0.94      0.97       553
         eng       0.98      0.97      0.98       603
         nbl       0.86      0.78      0.81       473
         nso       0.95      0.88      0.91       559
         sot       0.92      0.91      0.91       606
         ssw       0.91      0.82      0.87       481
         tsn       0.82      0.88      0.85       543
         tso       0.98      0.97      0.98       548
         ven       0.98      0.95      0.97       509
         xho       0.70      0.88      0.78       503
         zul       0.81      0.84      0.82       612

    accuracy                           0.90      5990
   macro avg       0.90      0.89      0.90      5990
weighted avg       0.90      0.90      0.90      5990



### __8.3 Random Forest__

In [22]:
from sklearn.ensemble import RandomForestClassifier

random_forest = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clr', RandomForestClassifier()),
              ])
random_forest.fit(X_train,y_train)
y_pred = random_forest.predict(X_test)

In [23]:
print(f"Accuracy is : {accuracy_score(y_pred,y_test)}")
print("\n\nClassification Report:\n\n", classification_report(y_test, y_pred))

Accuracy is : 0.9697829716193657


Classification Report:

               precision    recall  f1-score   support

         afr       1.00      0.99      0.99       553
         eng       0.99      1.00      0.99       603
         nbl       0.95      0.89      0.92       473
         nso       0.99      0.97      0.98       559
         sot       0.99      0.98      0.99       606
         ssw       0.97      0.95      0.96       481
         tsn       0.95      0.98      0.97       543
         tso       0.99      1.00      1.00       548
         ven       1.00      1.00      1.00       509
         xho       0.94      0.95      0.95       503
         zul       0.90      0.94      0.92       612

    accuracy                           0.97      5990
   macro avg       0.97      0.97      0.97      5990
weighted avg       0.97      0.97      0.97      5990



### __8.4 Support Vector Machine__

In [24]:
from sklearn.svm import SVC

svc = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clr', SVC(gamma='auto')),
              ])
svc.fit(X_train,y_train)
y_pred = svc.predict(X_test)


In [25]:
print(f"Accuracy is : {accuracy_score(y_pred,y_test)}")
print("\n\nClassification Report:\n\n", classification_report(y_test, y_pred))

Accuracy is : 0.10066777963272121


Classification Report:

               precision    recall  f1-score   support

         afr       0.00      0.00      0.00       553
         eng       0.10      1.00      0.18       603
         nbl       0.00      0.00      0.00       473
         nso       0.00      0.00      0.00       559
         sot       0.00      0.00      0.00       606
         ssw       0.00      0.00      0.00       481
         tsn       0.00      0.00      0.00       543
         tso       0.00      0.00      0.00       548
         ven       0.00      0.00      0.00       509
         xho       0.00      0.00      0.00       503
         zul       0.00      0.00      0.00       612

    accuracy                           0.10      5990
   macro avg       0.01      0.09      0.02      5990
weighted avg       0.01      0.10      0.02      5990



c:\Users\NeoSebanze\Desktop\June\5.automation_support\automated_nlp\env\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\NeoSebanze\Desktop\June\5.automation_support\automated_nlp\env\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\NeoSebanze\Desktop\June\5.automation_support\automated_nlp\env\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

### __8.4 KNN__

In [26]:
from sklearn.neighbors import KNeighborsClassifier

knn = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clr', KNeighborsClassifier(n_neighbors=3)),
              ])
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)

In [27]:
print(f"Accuracy is : {accuracy_score(y_pred,y_test)}")
print("\n\nClassification Report:\n\n", classification_report(y_test, y_pred))

Accuracy is : 0.9363939899833055


Classification Report:

               precision    recall  f1-score   support

         afr       0.96      0.98      0.97       553
         eng       0.94      0.99      0.96       603
         nbl       0.86      0.92      0.89       473
         nso       0.90      0.95      0.93       559
         sot       0.93      0.92      0.93       606
         ssw       0.96      0.93      0.95       481
         tsn       0.91      0.90      0.90       543
         tso       0.98      0.99      0.98       548
         ven       1.00      0.98      0.99       509
         xho       0.94      0.92      0.93       503
         zul       0.92      0.82      0.87       612

    accuracy                           0.94      5990
   macro avg       0.94      0.94      0.94      5990
weighted avg       0.94      0.94      0.94      5990



### __9.Tesing Unseen Data__ 

In [28]:
df_test.isnull().sum()

index    0
text     0
dtype: int64

In [29]:
# Clean_test
clean(df_test)

C:\Users\NeoSebanze\AppData\Local\Temp\ipykernel_17668\3637475032.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace("[^a-zA-Z-]", " ")


,index,text
0,1,mmasepala maemo kgethegileng letlelela kgato
1,2,uzakwaziswa ngokufaneleko nakungafuneka eminye imitlolo engezelelako ukuqedelela ukutloliswa kwesibawo sakho
2,3,tshivhumbeo tshi fana ngano vhathu
3,4,kube inja nelikati betingevakala kutsi titsini naticocisana
4,5,winste buitelandse valuta
...,...,...
5677,5678,mark your ballot private
5678,5679,kgetha bowena omi e mofani tirelo kgethilwego gems dsp gona tefelo mmogo omi wa tefelo mmogo phapano magareng hlwahlwa tirelo gems bego lefelet e nkabe omi it e gems goba persente tefelo mmogo melao setlamo gems
5679,5680,kopo etsa kgetho hloko hobane kgona fetola kgetho hara selemo ntle tumello boto batshwaredi
5680,5681,bokudi pmb morero lefella tlhahlobo sheba lefu lena leha jwalo tlameha fumana kalafo ditsing mmuso


In [30]:
df_sample

,index,lang_id
0,1,tsn
1,2,nbl


In [31]:
X_unseen = df_test['text']

In [ ]:
#y_predict = lr.predict(X_unseen) #linear regression
y_predict = mn.predict(X_unseen) #multinomial naive bayes
#y_predict = tree.predict(X_unseen) #decision tree
#y_predict = random_forest.predict(X_unseen) #decision tree
#y_predict = svc.predict(X_unseen) #svc
#y_predict = knn.predict(X_unseen) #knn


### __10.Final Model__

In [34]:
from sklearn import model_selection
import pickle

In [35]:
# storing best multi nomial naive bayes
pkl_file = "model.pkl"  

with open(pkl_file, 'wb') as file:  
    pickle.dump(mn, file)

In [36]:
# loading model
with open(pkl_file, 'rb') as file:  
    pickle_model = pickle.load(file)

### __11.Testing Model__

In [45]:
new_input = 'Lekker dag in die natuur, ons sien jou in die oggend'
# making predictions on unseen test dataset
predictions = pickle_model.predict([new_input])
predictions

array(['afr'], dtype='<U3')